# Manipulating data and metadata in cf-python

## Homepage https://ncas-cms.github.io/cf-python for background, tutorial, reference, and   installation

## Contents:

### 1. Read, inspect, write netCDF files
### 2. Subspace
### 3. Data
### 4. Calculate statistics
### 5. PP and UM datasets
### 6. What this course doesn't cover
----

# 1. Read, inspect and write files
https://ncas-cms.github.io/cf-python/function/cf.read.html

In [ ]:
import cf
cf.__version__

In [ ]:
f = cf.read('ncas_data/IPSL-CM5A-LR_r1i1p1_tas_n96_rcp45_mnth.nc')[0]

In [ ]:
f

In [ ]:
print(f)

https://ncas-cms.github.io/cf-python/method/cf.Field.dump.html

In [ ]:
f.dump()

### Properties

https://ncas-cms.github.io/cf-python/method/cf.Field.properties.html

In [ ]:
f.properties()

In [ ]:
f.get_property('project_id')

In [ ]:
f.set_property('project_id', 'banana')
f.get_property('project_id')

In [ ]:
f.del_property('project_id')
f.get_property('project_id')  # This should fail!

In [ ]:
f.get_property('project_id', 'UNSET')

https://ncas-cms.github.io/cf-python/method/cf.Field.get_property.html

In [ ]:
help(f.get_property)

### Shorthand for named CF properties
http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#attribute-appendix

In [ ]:
print(f.standard_name)
f.standard_name = 'banana'
print(f.standard_name)
del(f.standard_name)
f.standard_name = 'air_temperature'
print(f.standard_name)

### Reading many files

In [ ]:
fl = cf.read('ncas_data/data[2-7].nc')
fl

### A `FieldList` object inherits all of the usual Python `list` functionality 

In [ ]:
for x in fl:
    print('IDENTITY:', x.identity(), 'SHAPE:', x.shape, 'UNITS:', x.units)

### Select by list position

In [ ]:
g = fl[0]
g

In [ ]:
fl[4:]

### Select by metadata
https://ncas-cms.github.io/cf-python/tutorial.html#sorting-and-selecting-from-field-lists

In [ ]:
fl.select('air_temperature')

In [ ]:
fl.select('northward_wind')

In [ ]:
fl.select_by_units('km h-1')

In [ ]:
fl.select_by_units('km h-1', exact=False)

In [ ]:
import re
fl.select(re.compile('(east|north)ward_wind'))

# Write fields to a netCDF file
https://ncas-cms.github.io/cf-python/function/cf.write.html

In [ ]:
cf.write(f, 'new_file.nc')

https://ncas-cms.github.io/cf-python/method/cf.Field.equals.html

In [ ]:
g = cf.read('new_file.nc')[0]
f.equals(g)

# 2. Subspace a field


### Index-space: [square brackets] 
https://ncas-cms.github.io/cf-python/tutorial.html#subspacing-by-index

In [ ]:
print(f)

In [ ]:
print(f[0, 0, 0])

In [ ]:
print(f[0:6, :, :])

### Metadata-space: (`subspace` method)
https://ncas-cms.github.io/cf-python/tutorial.html#subspacing-by-metadata

In [ ]:
print(f)

In [ ]:
print(f.subspace(longitude=180))

#### `cf.lt(30)` is a "query" that means *less than 30*

https://ncas-cms.github.io/cf-python/tutorial.html#encapsulating-conditions

In [ ]:
print(f.subspace(latitude=cf.lt(30)))

#### `cf.wi(90, 270)` is a query that means *within the range [90, 270]*

In [ ]:
print(f.subspace(longitude=cf.wi(90, 270)))

In [ ]:
g = f.subspace(time=cf.dt('1965-11-16'))
print(g)

In [ ]:
# In-line images
%matplotlib inline
# Turn off warnings
#import warnings
#warnings.filterwarnings("ignore")

import cfplot as cfp
cfp.con(g)

#### `T` is shorthand for *time*

In [ ]:
print(f.subspace(T=cf.ge(cf.dt('1967-2-18'))))

In [ ]:
print(f.subspace(T=cf.month(4)))

In [ ]:
print(f.subspace(time=cf.dt('1965-11-16'), Y=cf.gt(30)))

# 3. The field's data

In [ ]:
f.data

#### Get the data as a `numpy` array

In [ ]:
print(type(f.array))

In [ ]:
f.array

In [ ]:
print(type(f.array))
f.array[-1, 3, -2]

In [ ]:
g = f.subspace[-1, 3, -2]
print(g)

In [ ]:
g.array

In [ ]:
x = f.copy()
x[0, 0, 0] = -999
x[0, 0, 0].array

In [ ]:
x.subspace[-1, ...] = 888
x.subspace[-1, ...].array

In [ ]:
import numpy
y = numpy.arange(145*192).reshape(145, 192)
print('Field shape:', x.shape)
print('Array shape:', y.shape)

In [ ]:
x[0, ...] = y
print(x[0, ...].array)

In [ ]:
print(x[1].array)

### Modify the data where a condition is met
https://ncas-cms.github.io/cf-python/tutorial.html#encapsulating-conditions

In [ ]:
print(f)

In [ ]:
f.data.stats()

#### Set values below 290 to missing data

In [ ]:
x = f.where(cf.lt(290), cf.masked)
x.data.stats()

In [ ]:
cfp.con(x.subspace[0])

### Manipulate the axes

In [ ]:
f.transpose(['X', 'T', 'Y'])

### Modifying the units

In [ ]:
f = cf.read('ncas_data/IPSL-CM5A-LR_r1i1p1_tas_n96_rcp45_mnth.nc')[0]
f.units, f.mean()

In [ ]:
f.units = 'degC'
f.units, f.mean()

In [ ]:
f.Units # Upper case "U" gives a units object that we can manipulate

In [ ]:
f.Units += 273.15
f.Units, f.units, f.mean()

### Field arithmetic

In [ ]:
f

In [ ]:
f.data.stats()

In [ ]:
g = f + 2
g

In [ ]:
g.data.stats() #min(), g.mean(), g.max()

In [ ]:
g = f - f
g

In [ ]:
g.data.stats()

In [ ]:
x = f.copy()
x.units = 'degC'
x.data

#### Subtract the celcius field from the Kelvin field and check that the result is zero 

In [ ]:
(f - x).mean()

In [ ]:
g = f * x
g

#### Find the anomalies relaitive to the first time

In [ ]:
first_time = f.subspace[0]
first_time = first_time.transpose(['Y', 'T', 'X'])
first_time

In [ ]:
g  = f - first_time
g

In [ ]:
g.data.stats()

In [ ]:
cfp.con(f.subspace(T=cf.contains(cf.dt('1962-06-04'))))
cfp.con(g.subspace(T=cf.contains(cf.dt('1962-06-04'))))

# 4. Statistical operations
https://ncas-cms.github.io/cf-python/analysis.html#statistical-collapses

In [ ]:
g = f.collapse('max')
g

In [ ]:
g.data

In [ ]:
g = f.collapse('T: mean')
print(g)
print('data values:\n', g.data)
print('\ntime bounds:\n', g.coord('T').bounds.dtarray)

In [ ]:
cfp.con(g)

#### Collapse multiple axes simultaneously

In [ ]:
g = f.collapse('X: Y: sd')
g

In [ ]:
cfp.lineplot(g)

#### Collapse an axis into groups, rather than a single value

In [ ]:
g = f.collapse('T: mean', group=cf.seasons())
print(g)

#### `cf.seasons()` is a list of queries, each of which defines a range of months

In [ ]:
cf.seasons()

#### By default, collpases are **not** weighted

In [ ]:
g = f.collapse('area: mean', weights='area') # Area mean for each time
g = g.collapse('T: max')                     # Time maxiumum of the area means
g.data
print(g)

### File aggregation

#### Create a sequence of files on disk, each of which contains one year

In [ ]:
f = cf.read('ncas_data/IPSL-CM5A-LR_r1i1p1_tas_n96_rcp45_mnth.nc')[0]
print(f)
for i in range(10):
    g = f.subspace[12*i:12*(i+1)]
    year = g.coord('T').year.array[0]
    new_file = 'air_temperature_'+str(year)+'.nc'
    cf.write(g, new_file)
    print('    ',new_file)

#### In Ipython `!` preceeeds a shell command

In [ ]:
!ls -o air_temperature_*.nc

In [ ]:
f2 = cf.read('air_temperature_*.nc')
print(f2)

In [ ]:
f.equals(f2[0])

In [ ]:
f3 = cf.read('air_temperature_*.nc', aggregate=False)
f3

# 5. PP and UM fields files

In [ ]:
x = cf.read('ncas_data/aaaaoa.pmh8dec.pp')
x

In [ ]:
print(x[1])

In [ ]:
cf.write(x, 'aaaaoa.pmh8dec.nc')

# 6. What this course doesn't cover

### Create new field constructs in memory
### Incorporate, and create, metadata stored in external files
### Read, write, and create data that have been compressed by convention (i.e. ragged or gathered arrays), whilst presenting a view of the data in its uncompressed form
### Perform histogram, percentile and binning operations on field constructs
### Apply convolution filters to field constructs
### Calculate derivatives of field constructs
### Create field constructs to create derived quantities (such as vorticity)